<a href="https://colab.research.google.com/github/EvertonFernandesJr/pomar-na-mao-processamentos/blob/main/Pomar_na_m%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import xml.etree.ElementTree as ET
import csv
import uuid
from datetime import datetime
import pytz

def kml_to_csv(folder_path: str, file: str):
    ns = {'kml': 'http://www.opengis.net/kml/2.2'}
    filename = file.split('.')[0]
    file_ext = file.split('.')[-1]

    if file_ext == 'kml':
        csv_filename = os.path.splitext(filename)[0] + '.csv'
        csv_path = os.path.join(folder_path, csv_filename)

        try:
            tree = ET.parse(file)
            root = tree.getroot()
            placemarks = root.findall(".//kml:Placemark", ns)

            with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.writer(csvfile)

                writer.writerow([
                    'id', 'created_at', 'longitude', 'latitude', 'gps_timestamp',
                    'mass', 'harvest', 'description', 'planting_date', 'user_id',
                    'life_of_the_tree', 'stick', 'broken_branch', 'vine_growing',
                    'burnt_branch', 'struck_by_lightning', 'drill', 'anthill',
                    'in_experiment', 'weeds_in_the_basin',
                    'fertilization_or_manuring', 'mites', 'thrips',
                    'empty_collection_box_near', 'variety', 'region','last_work', 'is_dead'
                ])

                for pm in placemarks:
                    name = pm.find('kml:name', ns)
                    coords = pm.find('.//kml:coordinates', ns)

                    if coords is not None:
                        coord_text = coords.text.strip()

                        # Se for polígono, pega só o primeiro vértice
                        if ' ' in coord_text:
                            first_coord = coord_text.split()[0]
                        else:
                            first_coord = coord_text

                        lon, lat, *_ = first_coord.split(',')

                        now = datetime.now(pytz.UTC)

                        writer.writerow([
                            str(uuid.uuid4()),
                            now.strftime('%Y-%m-%d %H:%M:%S.') + f'{int(now.microsecond / 10000):02d}+00',
                            lon.strip(),
                            lat.strip(),
                            1748620471648,
                            None,
                            None,
                            None,
                            None,
                            '680d9ad3-4069-4118-884d-505d18ad553c',
                            None,
                            False, False, False, False, False,
                            False, False, False, False, False,
                            False, False, False,
                            None,
                            'A',
                            None,
                            False
                        ])

            print(f"✔ Saved: {csv_filename}")

        except Exception as e:
            print(f"❌ Failed to process {filename}: {e}")

    else:
        print(f"❌ Invalid file extension: {file_ext}")

kml_to_csv('/content/', 'Arvores.kml')


✔ Saved: Arvores.csv


In [3]:
import csv
from google.colab import files

#1. Upload do arquivo
uploaded = files.upload()  # abre o seletor de arquivo no Colab
input_file = list(uploaded.keys())[0]
output_file = "output_" + input_file
print(f"Arquivo recebido: {input_file}")

#2. Colunas de saída
OUTPUT_COLUMNS = [
    "id", "created_at", "longitude", "latitude", "gps_timestamp", "mass",
    "harvest", "description", "planting_date", "life_of_the_tree", "stick",
    "broken_branch", "vine_growing", "burnt_branch", "struck_by_lightning",
    "drill", "anthill", "in_experiment", "weeds_in_the_basin",
    "fertilization_or_manuring", "mites", "thrips", "empty_collection_box_near",
    "variety", "region", "is_dead", "is_new", "non_existent", "frost",
    "flowers", "buds", "dehydrated", "updated_at"
]

#3. Detecta colunas X e Y
with open(input_file, newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    orig_fields = reader.fieldnames

x_col = next((c for c in orig_fields if c.strip().lower() in ('x', 'longitude', 'lon', 'lng')), None)
y_col = next((c for c in orig_fields if c.strip().lower() in ('y', 'latitude', 'lat')), None)

if not x_col or not y_col:
    raise ValueError(f"Colunas X/Y não encontradas. Colunas disponíveis: {orig_fields}")

print(f"Mapeando  '{x_col}' → longitude  |  '{y_col}' → latitude")

BOOLEAN_FALSE_FIELDS = {
    "stick", "broken_branch", "vine_growing", "burnt_branch",
    "struck_by_lightning", "drill", "anthill", "in_experiment",
    "weeds_in_the_basin", "fertilization_or_manuring", "mites", "thrips",
    "empty_collection_box_near", "is_dead", "is_new", "non_existent",
    "frost", "flowers", "buds", "dehydrated"
}

#4. Conversão
with open(input_file, newline='', encoding='utf-8') as infile, \
     open(output_file, 'w', newline='', encoding='utf-8') as outfile:

    reader = csv.DictReader(infile)
    writer = csv.DictWriter(outfile, fieldnames=OUTPUT_COLUMNS)
    writer.writeheader()

    for i, row in enumerate(reader, start=1):
        new_row = {col: None for col in OUTPUT_COLUMNS}
        new_row["id"]        = None
        new_row["longitude"] = row.get(x_col, '').strip()
        new_row["latitude"]  = row.get(y_col, '').strip()
        new_row["mass"] = 0

        for field in BOOLEAN_FALSE_FIELDS:
          new_row[field] = False
        writer.writerow(new_row)


print(f"Conversão concluída! Arquivo gerado: {output_file}")

# ── 5. Download automático do resultado ───────────────────────────────────────
files.download(output_file)

Saving TestePomar7.csv to TestePomar7 (1).csv
Arquivo recebido: TestePomar7 (1).csv
Mapeando  'X' → longitude  |  'Y' → latitude
Conversão concluída! Arquivo gerado: output_TestePomar7 (1).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os
import xml.etree.ElementTree as ET
import csv
import uuid
from datetime import datetime
import pytz

def kml_to_csv(folder_path: str, file: str):
    ns = {'kml': 'http://www.opengis.net/kml/2.2'}
    filename = file.split('.')[0]
    file_ext = file.split('.')[-1]

    if file_ext == 'kml':
        csv_filename = os.path.splitext(filename)[0] + '.csv'
        csv_path = os.path.join(folder_path, csv_filename)

        try:
            tree = ET.parse(file)
            root = tree.getroot()
            placemarks = root.findall(".//kml:Placemark", ns)

            with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.writer(csvfile)

                writer.writerow([
                    'id', 'created_at', 'longitude', 'latitude', 'gps_timestamp',
                    'mass', 'harvest', 'description', 'planting_date', 'user_id',
                    'life_of_the_tree', 'stick', 'broken_branch', 'vine_growing',
                    'burnt_branch', 'struck_by_lightning', 'drill', 'anthill',
                    'in_experiment', 'weeds_in_the_basin',
                    'fertilization_or_manuring', 'mites', 'thrips',
                    'empty_collection_box_near', 'variety', 'region', 'last_work', 'is_dead'
                ])

                for pm in placemarks:
                    name = pm.find('kml:name', ns)
                    coords = pm.find('.//kml:coordinates', ns)

                    if coords is not None:
                        coord_text = coords.text.strip()
                        lon, lat, *_ = coord_text.split(',')
                        zona = name.text[-1]
                        if name.text[-1].islower():
                          zona = 'FEAGRI'


                        now = datetime.now(pytz.UTC)

                        writer.writerow([
                            str(uuid.uuid4()),
                            now.strftime('%Y-%m-%d %H:%M:%S.') + f'{int(now.microsecond / 10000):02d}+00',
                            lon.strip(),
                            lat.strip(),
                            1748620471648,
                            100,
                            '2024',
                            name.text if name is not None else '',
                            now.strftime('%Y-%m-%d %H:%M:%S.') + f'{int(now.microsecond / 10000):02d}+00',
                            '680d9ad3-4069-4118-884d-505d18ad553c',
                            'Primeira',
                            False, False, False, False, False,
                            False, False, False, False, False,
                            False, False, False,
                            'Clássica (Bengal)',
                            zona
                        ])

            print(f"✔ Saved: {csv_filename}")

        except Exception as e:
            print(f"❌ Failed to process {filename}: {e}")

    else:
        print(f"❌ Invalid file extension: {file_ext}")

# Exemplo de uso:
kml_to_csv('/content/', 'Arvores.kml')


✔ Saved: Zonas.csv
